In [ ]:
%run -i ../startup.py
ENTITY_TYPE = 'release-group'
endpoint='http://data.bnf.fr/sparql'

## Release Groups from BNF

In [ ]:
# linked to MB RG
bnf_entity_count(ENTITY_TYPE)

In [ ]:
links_from_bnf = sparql(f"""
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT (?rg AS ?bnf) ?mbid ?name
WHERE {{
  ?rg skos:exactMatch ?mbid .
  ?rg skos:prefLabel ?name .
  FILTER (regex (?mbid, 'musicbrainz.org/release-group/'))
}}
""", endpoint='http://data.bnf.fr/sparql')

print('Count:', len(links_from_bnf))
display_df(links_from_bnf.head())

In [ ]:
links_to_add_to_mb = links_from_bnf
links_to_add_to_mb['edit_link'] = links_to_add_to_mb.apply(
    mb_releasegroup_edit_bnf_link, axis=1)

print('Count:', len(links_to_add_to_mb))
display_df(links_to_add_to_mb.head())

## Report

In [ ]:
import jinja2

template = jinja2.Template("""
<!doctype html>

<html lang="en">
  <head>
    <meta charset="utf-8">
    <title>Alignment of MusicBrainz and BNF Release Groups</title>
    <link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous">
  </head>

  <body style="margin: 20px;">
    <h1>Alignment of MusicBrainz and BNF Release Groups</h1>

    <p>Latest MB database update: {{ MB_DATABASE_VERSION }}</p>
    <p>Latest update: {{ date.today().isoformat() }}</p>

    <ol>
      <li>
        <a href="#bnf2mb">Add missing BNF links to MusicBrainz</a>
        ({{ links_to_add_to_mb.shape[0] }} rows)
      </li>
    </ol>
    
    <h2 id="bnf2mb">Add missing BNF links to MusicBrainz</h2>
    {{ df_to_html(links_to_add_to_mb) }}
    
  </body>
</html>
""")

with open('../docs/bnf-releasegroups-report.html', 'w') as f:
    f.write(template.render(**globals())
            .replace('&lt;', '<').replace('&gt;', '>')
            .replace('class="dataframe"', 'class="table table-striped table-hover table-sm"')
            .replace('thead', 'thead class="thead-light"'))